# Code

EV aggregator implementation based on State Space Model.

### EV SSM Class

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from ev_ssm import ev_ssm
from ev_ssm import r_agc_sev
from tqdm import tqdm

In [2]:
# --- Control Error ---
Pil = []
Prl = []
Pcl = []
error = []
t0 = 10.8
for Pi in np.arange(0.5, 8, 0.2):
        sse = ev_ssm(ts=t0, N=10000, step=1, tp=40,
                lr=0.1, lp=40, seed=2022, name="EVA")
        sse.load_A("Aest.csv")

        c0 = sse.ev.c
        sse.run(tf=t0 + 1.5/3600, Pi=Pi,
                is_updateA=False, is_rstate=True,
                is_test=False, disable=False)

        [u, v, us, vs] = sse.uv
        [A, B, C, D] = [sse.A, sse.B, sse.C, sse.D]

        Pil.append(Pi)
        Prl.append(sse.Pr)
        Pcl.append(sse.Prc)
        error.append((sse.Prc - sse.Pr) / sse.Pr)

EVA: ts=10.8[H], 10000 EVs, Total Q=250.12 MWh
Online 1613, Q=40.54 MWh, SoC=0.968
Power(MW): Pt=-0.5954, Pc=-0.5954, Pd=-0.0
Ctrl: 0=1513; 1=100; 
EVA: Load A from Aest.csv.
EVA MCS: 100%|██████████| 2/2 [00:00<00:00,  4.64it/s]
EVA: ts=10.8[H], 10000 EVs, Total Q=250.12 MWh
Online 1613, Q=40.54 MWh, SoC=0.968
Power(MW): Pt=-0.5954, Pc=-0.5954, Pd=-0.0
Ctrl: 0=1513; 1=100; 
EVA: Load A from Aest.csv.
EVA MCS: 100%|██████████| 2/2 [00:00<00:00,  5.17it/s]
EVA: ts=10.8[H], 10000 EVs, Total Q=250.12 MWh
Online 1613, Q=40.54 MWh, SoC=0.968
Power(MW): Pt=-0.5954, Pc=-0.5954, Pd=-0.0
Ctrl: 0=1513; 1=100; 
EVA: Load A from Aest.csv.
EVA MCS: 100%|██████████| 2/2 [00:00<00:00,  2.57it/s]
EVA: ts=10.8[H], 10000 EVs, Total Q=250.12 MWh
Online 1613, Q=40.54 MWh, SoC=0.968
Power(MW): Pt=-0.5954, Pc=-0.5954, Pd=-0.0
Ctrl: 0=1513; 1=100; 
EVA: Load A from Aest.csv.
EVA MCS: 100%|██████████| 2/2 [00:00<00:00,  4.87it/s]
EVA: ts=10.8[H], 10000 EVs, Total Q=250.12 MWh
Online 1613, Q=40.54 MWh, SoC=0.9

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.plot(Pil, Prl, label="Signal")
ax.plot(Pil, Pcl, label="Response")
ax.set_xlabel("Input power [p.u.]")
ax.set_ylabel("Output power [p.u.]")
ax.set_title(f"Time={sse.tss[0]} [H]")
ax.set_xlim([0.5, 7.7])
ax.grid(True)
ax.legend()
ax2 = ax.twinx()
ax2.plot(Pil, error, label='Error', color='red')

In [ ]:
sse.ev[sse.ev.agc != 0].shape

In [ ]:
sse.ev[sse.ev.u == 1].shape

In [ ]:
sse.ev[(sse.ev.c0 != sse.ev.c) & (sse.ev.u==1)][['c0', 'c', 'sx']]

In [ ]:
# --- Single Response ---

Pi = 8

sse = ev_ssm(ts=t0, N=10000, step=1, tp=40,
        lr=0.1, lp=40, seed=2022, name="EVA")
sse.load_A("Aest.csv")

r0 = sse.rtab.values.reshape(-1)
x0 = sse.x0
sse.run(tf=t0 + 1.5/3600, Pi=Pi,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=True)
r = sse.rtab.values.reshape(-1)
x = sse.x0

[u, v, us, vs] = sse.uv
[A, B, C, D] = [sse.A, sse.B, sse.C, sse.D]

print(sse.Pr)
print(sse.Prc)
print(sse.Prc - sse.Pr)

In [ ]:
# --- Replay ---
# Replay EV control signal from recorded data
# Replay 1 hour data may need 0.5 hour
# Replay is not accurate

sse_out = pd.read_csv('sse_out.csv')
sse_out_s = sse_out.iloc[0:200].reset_index(drop=True)

t0 = sse_out_s['time'].iloc[0]
sse = ev_ssm(ts=t0, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA")
sse.load_A("Aest.csv")

ev_soc = pd.DataFrame(columns=range(sse_out_s.shape[0]))
ev_agc = pd.DataFrame(columns=range(sse_out_s.shape[0]))

for row in tqdm(range(sse_out_s.shape[0]), desc=f'{sse.name} Replay'):
    sse.run(tf=sse_out_s.time.iloc[row], Pi=sse_out_s.Pr.iloc[row],
            is_updateA=False, is_rstate=True,
            is_test=False, disable=True)
    ev_soc[row] = sse.ev.soc
    ev_agc[row] = sse.ev.agc

In [ ]:
sse.Prc

In [ ]:
sse.Pr

In [ ]:
sse.plot_agc()

In [ ]:
sse.Prc

In [ ]:
sse.Pr

In [ ]:
sse.y

In [ ]:
sse.y0

In [ ]:
sse.rtab.values.reshape(-1).round(3)

In [ ]:
sse.r_state()
sse.x0.round(3)

In [ ]:
sse.plot()